In [61]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
apiKey = os.getenv("HKBU_API_KEY")
basicUrl = os.getenv("HKBU_BASIC_URL")
if apiKey is None:
    raise ValueError("HKBU_API_KEY not found in environment variables")

In [62]:
gemini_models = [
    {
        "model": "gemini-1.5-pro",
        "api-version": "002"
    },
    {
        "model": "gemini-1.5-flash",
        "api-version": "002"
    }
]


## Define Function

In [63]:
def add_function_declaration(name: str, description: str, parameters: dict):
    """
    Helper function to create a function declaration.
    """
    return {
        "name": name,
        "description": description,
        "parameters": parameters
    }


In [ ]:

def Gemini(
    message: str,
    model_name: str = "gemini-1.5-flash",
    temperature: float = 0.5,
    maxOutputTokens: int = 10,
    response_mime_type: str = "application/json",
    response_schema: dict = None,
    function_declarations: list = None,  # List of function declarations
    tool_choice: str = "auto",
    allowed_function_names: list = None,
):
    # Find the model in the gemini_models list
    model_info = next((model for model in gemini_models if model["model"] == model_name), None)
    
    if not model_info:
        raise ValueError(f"Model {model_name} not found in gemini_models list")
    
    api_version = model_info['api-version']
    
    url = f"{basicUrl}/deployments/{model_name}/generate_content?api-version={api_version}"
    headers = {
        'Content-Type': 'application/json',
        'accept': 'application/json',
        'api-key': apiKey
    }
    
    # Define the contents with the user's message
    contents = [{"role": "user", "parts": [{"text": message}]}]
    
    # Define the payload with the generation config and optional response schema
    payload = {
        'contents': contents,
        'generationConfig': {
            'maxOutputTokens': maxOutputTokens,
            'temperature': temperature,
            "response_mime_type": response_mime_type,
        },
        'stream': False,
    }
    
    if function_declarations:
        # Validate function_declarations
        if not isinstance(function_declarations, list) or not all(isinstance(func, dict) for func in function_declarations):
            raise ValueError("function_declarations must be a list of dictionaries.")
        
        if allowed_function_names is None:
            allowed_function_names = [func['name'] for func in function_declarations]
        
        # Ensure allowed_function_names are valid
        declared_function_names = [func['name'] for func in function_declarations]
        if not set(allowed_function_names).issubset(set(declared_function_names)):
            raise ValueError("allowed_function_names contains functions not declared in function_declarations.")
        
        # Wrap function_declarations in the required tools structure
        tools = [{
            "function_declarations": function_declarations
        }]
        
        # Configure tools and tool_config
        payload["tools"] = tools
        payload["tool_choice"] = tool_choice
        payload["generationConfig"].pop("response_mime_type", None)
        payload["generationConfig"].pop("response_schema", None)
        payload["tool_config"] = {
            "function_calling_config": {
                "mode": "ANY",
                "allowed_function_names": allowed_function_names
            }
        }
    
    if response_schema:
        payload['generationConfig']['response_schema'] = response_schema
    
    # Make the POST request to the API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response status code
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        print(f"Response: {response.text}")
        return None


In [ ]:

# Example usage
message = "What is the weather like in Boston?"

# Use the helper function to define the function declaration
function_declarations = [
    add_function_declaration(
        name="get_current_weather",
        description="Get the current weather in a given location",
        parameters={
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
                }
            },
            "required": ["location"]
        }
    )
]

response = Gemini(
    message, 
    model_name="gemini-1.5-flash",
    temperature=0.5,
    maxOutputTokens=100,
    function_declarations=function_declarations,
    tool_choice="auto",
    allowed_function_names=["get_current_weather"]
)

print(json.dumps(response, indent=2))

{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "name": "get_current_weather",
              "args": {
                "location": "Boston, MA"
              }
            }
          }
        ]
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.06301789151297675,
      "index": 0
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 50,
    "candidatesTokenCount": 9,
    "totalTokenCount": 59,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 50
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 9
      }
    ]
  },
  "modelVersion": "gemini-1.5-flash-002",
  "createTime": "2025-02-22T03:26:36.696432Z",
  "responseId": "7EO5Z_DAKv-v-dIP27vK2Ag"
}


In [66]:
def get_current_weather(location):
    if location == "San Francisco":
        return '{ "location": "San Francisco, CA", "temperature": 65, "description": "Sunny", "icon": "sunny", "humidity": 70, "wind": { "speed": 12, "direction": "W" } }'
    elif location == "Boston, MA":
         return '{ "location": "Boston, MA", "temperature": 38, "description": "Partly Cloudy", "icon": "partly-cloudy", "humidity": 65, "wind": { "speed": 10, "direction": "NW" } }'
    else:
        return '{ "location": "Unknown", "temperature": 0, "description": "Unknown", "icon": "unknown", "humidity": 0, "wind": { "speed": 0, "direction": "N" } }'

In [67]:
print(f'Called function: {response["candidates"][0]["content"]['parts'][0]["functionCall"]["name"]}')
print(f'Function args: {response["candidates"][0]["content"]['parts'][0]["functionCall"]["args"]}')

Called function: get_current_weather
Function args: {'location': 'Boston, MA'}


In [68]:
# Check the function name that the model responded with, and make an API call to an external system
if (response["candidates"][0]["content"]['parts'][0]["functionCall"]["name"] == "get_current_weather"):
    # Extract the arguments to use in your API call
    location = response["candidates"][0]["content"]['parts'][0]["functionCall"]["args"]["location"]

    # Here you can use your preferred method to make an API request to fetch the current weather, for example:
    # api_response = requests.post(weather_api_url, data={"location": location})

    # In this example, we'll use synthetic data to simulate a response payload from an external API
    api_response = get_current_weather(location)
    display(json.loads(api_response))

{'location': 'Boston, MA',
 'temperature': 38,
 'description': 'Partly Cloudy',
 'icon': 'partly-cloudy',
 'humidity': 65,
 'wind': {'speed': 10, 'direction': 'NW'}}